In [2]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import time

start_time = time.time()
# Function to convert SMILES to molecular fingerprint
def smiles_to_ECFP(smiles,n_bits):
    if smiles is not None:
        mol = Chem.MolFromSmiles(smiles)    
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=n_bits)
        return np.array(fp)
    else:
        return np.zeros(n_bits)
        
# Get fingerprints of a reaction 
def get_fingerprint_features(df, components): 
    components_fp_list= [] 
    for i in components:
        component_fp = np.array([smiles_to_ECFP(smiles,1024) for smiles in df[i]])
        components_fp_list.append(component_fp)
    components_fp=np.hstack(components_fp_list)
    return components_fp

# read information from excel files to generate a dataframe    
df1 = pd.read_excel(r'I:\acceptor_HTE.xlsx',sheet_name='substrates_train').drop(columns=['ligands','catalysts'])
df2 = pd.read_excel(r'I:\acceptor_HTE.xlsx',sheet_name='substrates_test').drop(columns=['ligands','catalysts'])

# Encode reactions and generate training dateset and test dataset
column_list=df1.columns.tolist()
components=column_list[3:6]
train_data=pd.DataFrame(get_fingerprint_features(df1, components))
train_data['ee']=df1['ee']
train_data['catalysts_index']=df1['catalysts_index']
test_data=pd.DataFrame(get_fingerprint_features(df2, components))
test_data['ee']=df2['ee']
test_data['catalysts_index']=df2['catalysts_index']

# Construction of AutoGluon model and do prediction
predictor = TabularPredictor(label='ee').fit(train_data)
predictor.predict(test_data.drop(columns=['ee'])).to_excel(r'I:\acceptor_HTE_prediction.xlsx')
test_metrics=predictor.evaluate(test_data, silent=True)
train_metrics=predictor.evaluate(train_data, silent=True)
end_time = time.time()
print('The metric of training dataset:', train_metrics)
print('The metric of test dataset:', test_metrics)
print(f'{end_time-start_time}seconds')

No path specified. Models will be saved in: "AutogluonModels\ag-20250401_091925"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20250401_091925"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.9.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          8
Memory Avail:       

The metric of training dataset: {'root_mean_squared_error': -0.12661806755617352, 'mean_squared_error': -0.016032135031659723, 'mean_absolute_error': -0.08456279381804707, 'r2': 0.8207906458286531, 'pearsonr': 0.9068968618585792, 'median_absolute_error': -0.054830049051807106}
The metric of test dataset: {'root_mean_squared_error': -0.12700336761654998, 'mean_squared_error': -0.01612985538594454, 'mean_absolute_error': -0.10093877098944568, 'r2': 0.7716658768152936, 'pearsonr': 0.8870633882274118, 'median_absolute_error': -0.08979353867640877}
14.69510006904602seconds
